In [ ]:
import faiss
import numpy as np
import pandas as pd
import openai
import pickle  # For saving text data
from typing import List

# Initialize OpenAI client (Replace API key with your own)
client = openai.OpenAI(api_key="")

# Dictionary for non-ASCII material properties
material_properties_with_symbols = {
    "Yield Strength": ["σᵧ"],
    "Ultimate Tensile Strength": ["σₘₐₓ"],
    "Shear Strength": ["τ"],
    "Strain": ["ε"],
    "Stress": ["σ"],
    "Fatigue": ["Δσ"],
    "Creep": ["ε̇"],
    "Poisson's Ratio": ["ν"],
    "Damping Capacity": ["ζ"],
    "Thermal Expansion Coefficient": ["α"],
    "Thermal Conductivity": ["λ"],
    "Seebeck Coefficient": ["S", "α"],
    "Permittivity": ["ε"],
    "Charge Transfer Impedance": ["Z"],
    "Band Gap": ["E₉"],
    "Work Function": ["Φ"],
    "Dielectric Constant": ["κ", "εᵣ"],
    "Hall Coefficient": ["R_H"],
    "Electron Mean Free Path": ["λ"],
    "Magnetization": ["M"],
    "Saturation Magnetization": ["Mₛ"],
    "Coercivity": ["H_c"],
    "Curie Temperature": ["T_c"],
    "Magnetic Susceptibility": ["χ"],
    "Remanence": ["M_r"],
    "Magnetostriction": ["λ_s"],
    "Exchange Bias Field": ["H_ex"],
    "Magnetic Permeability": ["μ"],
    "Friction": ["μ"],
    "Core Loss": ["P_c"],
    "Elastic Moduli": ["E", "G", "B"],
    "Interfacial Energy": ["γ"],
    "Vacancy Formation Energy": ["E_v"],
    "Cohesive Energy": ["E_coh"],
    "Crack-Growth Rates": ["da/dN"],
    "Crystallographic Orientation": ["θ"],
    "Lattice Constants": ["a", "b", "c", "α", "β", "γ"],
    "Mean-Square Atomic Displacements": ["⟨u²⟩"],
    "Phase Transition Temperatures": ["T_t"],
    "Phase Transition": ["ΔH"],
    "Elasticity": ["E", "G", "B", "ν"],
    "Stress and Strain": ["σ", "ε"],
    "Electronegativity": ["χ"],
    "Zener Ratio": ["A"]
}

# Dictionary to convert non-ASCII symbols to ASCII equivalents
symbol_to_english = {
    "α": "alpha",
    "β": "beta",
    "γ": "gamma",
    "δ": "delta",
    "ε": "epsilon",
    "ζ": "zeta",
    "η": "eta",
    "θ": "theta",
    "ι": "iota",
    "κ": "kappa",
    "λ": "lambda",
    "μ": "mu",
    "ν": "nu",
    "ξ": "xi",
    "ο": "omicron",
    "π": "pi",
    "ρ": "rho",
    "σ": "sigma",
    "τ": "tau",
    "υ": "upsilon",
    "φ": "phi",
    "χ": "chi",
    "ψ": "psi",
    "ω": "omega",
    
    # Capital Greek letters
    "Α": "Alpha",
    "Β": "Beta",
    "Γ": "Gamma",
    "Δ": "Delta",
    "Ε": "Epsilon",
    "Ζ": "Zeta",
    "Η": "Eta",
    "Θ": "Theta",
    "Ι": "Iota",
    "Κ": "Kappa",
    "Λ": "Lambda",
    "Μ": "Mu",
    "Ν": "Nu",
    "Ξ": "Xi",
    "Ο": "Omicron",
    "Π": "Pi",
    "Ρ": "Rho",
    "Σ": "Sigma",
    "Τ": "Tau",
    "Υ": "Upsilon",
    "Φ": "Phi",
    "Χ": "Chi",
    "Ψ": "Psi",
    "Ω": "Omega",
    
    # Subscripts and special cases
    "σᵧ": "sigma_y",
    "σₘₐₓ": "sigma_max",
    "Δσ": "delta_sigma",
    "ε̇": "epsilon_dot",
    "εᵣ": "epsilon_r",
    "E₉": "E_g",
    "H_c": "H_c",
    "T_c": "T_c",
    "Mₛ": "M_s",
    "M_r": "M_r",
    "H_ex": "H_ex",
    "P_c": "P_c",
    "E_v": "E_v",
    "E_coh": "E_coh",
    "da/dN": "da_dN",
    "⟨u²⟩": "u_squared",
    "T_t": "T_t",
    "ΔH": "delta_H",
    
    # Multi-letter cases
    "a, b, c": "a_b_c",
    "α, β, γ": "alpha_beta_gamma",
    
    # Miscellaneous
    "S": "S",
    "Z": "Z",
    "R_H": "R_H",
    "M": "M",
    "G": "G",
    "B": "B",
    "A": "A",
    "E": "E"
}


# Function to replace symbols with ASCII equivalents
def replace_symbols(text):
    for symbol, ascii_rep in symbol_to_english.items():
        text = text.replace(symbol, ascii_rep)
    return text

In [ ]:
class VectorStore:
    def __init__(self, embedding_dim=1536, index_file="vector_store.index", metadata_file="metadata.pkl"):
        self.embedding_dim = embedding_dim
        self.index_file = index_file
        self.metadata_file = metadata_file
        self.text_data = []  # Stores original text entries

        # Load existing FAISS index if available
        try:
            self.index = faiss.read_index(self.index_file)
            with open(self.metadata_file, "rb") as f:
                self.text_data = pickle.load(f)
            print("Loaded existing vector store.")
        except:
            self.index = faiss.IndexFlatIP(embedding_dim)  # Use Inner Product (IP) for cosine similarity
            print("Created new vector store.")

    def get_embedding(self, text: str) -> np.ndarray:
        """Generate embedding for a given text."""
        response = client.embeddings.create(input=[text], model="text-embedding-ada-002")
        embedding = np.array(response.data[0].embedding, dtype=np.float32)
        return embedding / (np.linalg.norm(embedding) + 1e-10)  # Normalize for cosine similarity

    def add_texts(self, texts: List[str]):
        """Add texts to the vector store with normalization."""
        embeddings = np.array([self.get_embedding(text) for text in texts], dtype=np.float32)
        self.index.add(embeddings)
        self.text_data.extend(texts)

        # Save index and metadata
        faiss.write_index(self.index, self.index_file)
        with open(self.metadata_file, "wb") as f:
            pickle.dump(self.text_data, f)
        print("Vector store updated and saved.")

    def search(self, query: str, top_k=3) -> List[str]:
        """Search for top_k most similar texts using dot product."""
        query_embedding = self.get_embedding(query).reshape(1, -1)  # Normalize query
        distances, indices = self.index.search(query_embedding, top_k)
        return [self.text_data[i] for i in indices[0] if i < len(self.text_data)]
    
    def print_all_entries(self):
        """Print all stored text entries in the vector store."""
        for i, text in enumerate(self.text_data):
            print(f"{i}: {text}")
            print()
            print()

In [ ]:
import pandas as pd
import ast

# Load CSV
df = pd.read_csv("final_properties_with_descriptions.csv")

# Ensure "Variations" column is treated as a list
def safe_eval_list(value):
    try:
        return ast.literal_eval(value) if isinstance(value, str) else value
    except:
        return []

df["Variations"] = df["Variations"].apply(safe_eval_list)

# Extend "Variations" with mapped English representations
new_rows = []
for _, row in df.iterrows():
    prop = row["Property"]
    variations = row["Variations"]
    
    if prop in material_properties_with_symbols:
        ascii_variations = [symbol_to_english[s] for s in material_properties_with_symbols[prop] if s in symbol_to_english]
        variations = list(set(variations + ascii_variations))
    
    # Split variations into chunks of 1
    for i in range(0, len(variations)):
        new_row = row.copy()
        new_row["Variations"] = variations[i:i + 1]
        new_rows.append(new_row)

# Create new DataFrame
df_expanded = pd.DataFrame(new_rows)

# Save updated CSV
df_expanded.to_csv("updated_properties_with_descriptions.csv", index=False)


# Example usage
# query = "Ecorr and i corr of alloys"
# normalized_query = normalize_query(query)
# print(normalized_query)


Ecorr and i corr of alloys


In [ ]:
# Load CSV and process data
csv_file = "updated_properties_with_descriptions.csv"
df = pd.read_csv(csv_file, encoding="utf-8")

# Ensure the necessary columns exist
required_columns = ["Property", "Type", "Description"]
if not all(col in df.columns for col in required_columns):
    raise ValueError("CSV is missing required columns: 'Property Name', 'Type', 'Description'")


# Combine columns into a single string
df["text"] = df["Property"]+" "+df["Variations"]
df = df.drop_duplicates(subset=["text"])
# Initialize vector store
store = VectorStore()

# Add data to vector store
store.add_texts(df["text"].tolist())


Created new vector store.
Vector store updated and saved.

Top 3 similar properties:
- Repassivation Potential ['Erp']
- Repassivation Potential ['E_rp']
- Pitting Potential ['Epit']


In [4]:
# Function to convert non-ASCII characters in a query string
def normalize_query(query):
    for symbol, ascii_rep in symbol_to_english.items():
        query = query.replace(symbol, ascii_rep)
    return query

In [9]:
# Test retrieval
query = "εp (%)"
normalized_query=normalize_query(query)
print(normalized_query)
results = store.search(normalized_query,top_k=10)

print("\nTop 10 similar properties:")
for r in results:
    print("-", r)

epsilonp (%)

Top 10 similar properties:
- Strain ['epsilon']
- Permittivity ['epsilon']
- Passivation Potential ['Epass']
- Stress and Strain ['epsilon']
- Repassivation Potential ['Erp']
- Repassivation Potential ['E_rp']
- Pitting Potential ['Epit']
- Dielectric Constant ['epsilon_r']
- Passivation Potential ['E_pass']
- Entropy ['Entropy']


In [5]:
import csv
import io

def convert_to_csv(table: str):
    # Split rows by newline
    rows = table.strip().split("\n")

    # Skip the title (first row)
    data_rows = rows[:]  

    # Convert tab-separated values to CSV format
    output = io.StringIO()
    writer = csv.writer(output)
    
    row_count = 0
    col_count = 0

    for row in data_rows:
        columns = row.split("\t")
        writer.writerow(columns)

        if len(columns) > 1:  # Ensure it's a valid data row
            row_count += 1
            col_count = max(col_count, len(columns))  # Track max columns

    return output.getvalue(), row_count, col_count


In [3]:
output=convert_to_csv("""Table 1. Chemical compositions (in at.%), density, ρ, and Vickers microhardness, Hv, of the alloys studied in this work.

Alloy	Cr	Nb	Ti	V	Zr	ρ (g cm−3)	ρmix (g cm−3)	HV (GPa)
NbTiVZr	–	28.3	24.5	23.0	24.2	6.52	6.50	3.29 ± 0.03
NbTiV2Zr	–	22.6	19.4	37.2	20.8	6.34	6.38	2.99 ± 0.03
CrNbTiZr	24.6	26.7	23.9	–	24.8	6.67	6.70	4.10 ± 0.04
CrNbTiVZr	20.2	20.0	19.9	19.6	20.3	6.57	6.52	4.72 ± 0.05""")
print(output)

('"Table 1. Chemical compositions (in at.%), density, ρ, and Vickers microhardness, Hv, of the alloys studied in this work."\r\n""\r\nAlloy,Cr,Nb,Ti,V,Zr,ρ (g cm−3),ρmix (g cm−3),HV (GPa)\r\nNbTiVZr,–,28.3,24.5,23.0,24.2,6.52,6.50,3.29 ± 0.03\r\nNbTiV2Zr,–,22.6,19.4,37.2,20.8,6.34,6.38,2.99 ± 0.03\r\nCrNbTiZr,24.6,26.7,23.9,–,24.8,6.67,6.70,4.10 ± 0.04\r\nCrNbTiVZr,20.2,20.0,19.9,19.6,20.3,6.57,6.52,4.72 ± 0.05\r\n', 5, 9)


In [6]:
store = VectorStore()
import csv
import io

def process_csv_and_retrieve_properties(csv_string: str, store, similarity_threshold=0.3, num_properties=10) -> set:
    """
    Processes CSV data, retrieves top properties from the vector store, 
    and filters them based on a cosine similarity threshold.

    :param csv_string: The CSV-formatted string.
    :param store: The vector store object with a search function.
    :param similarity_threshold: Minimum cosine similarity to retain a property.
    :return: A set of top 10 unique retrieved properties sorted by similarity score.
    """
    csv_reader = csv.reader(io.StringIO(csv_string))
    shortlisted_properties = []  # Stores (property_name, similarity) pairs

    for row in csv_reader:
        for cell in row:
            # Ensure the cell is a non-empty string
            if not isinstance(cell, str) or not cell.strip():
                continue  # Skip empty cells
            
            normalized_query = normalize_query(cell.strip())  # Normalize the query

            try:
                query_embedding = store.get_embedding(normalized_query).reshape(1, -1)
            except Exception as e:
                print(f"Skipping '{cell}' due to embedding error: {e}")
                continue  # Skip problematic queries

            # Perform search with similarity scores
            distances, indices = store.index.search(query_embedding, 20)

            # Store only top 3 unique properties per query
            query_top_properties = []
            seen_query_properties = set()
            # print(normalized_query)
            for idx, similarity in zip(indices[0], distances[0]):
                if idx < len(store.text_data) and similarity >= similarity_threshold:
                    property_name = store.text_data[idx].split(" [")[0]
                    # print(property_name,similarity)
                    if property_name not in seen_query_properties:
                        query_top_properties.append((property_name, similarity))
                        seen_query_properties.add(property_name)

                    if len(query_top_properties) == 3:  #top 3 per query
                        break

            # Add shortlisted properties for final selection
            shortlisted_properties.extend(query_top_properties)

    # Sort all shortlisted properties by similarity in descending order
    shortlisted_properties.sort(key=lambda x: x[1], reverse=True)

    # Retain only the top 10 unique properties
    final_top_properties = []
    seen_properties = set()
    
    for property_name, similarity in shortlisted_properties:
        if property_name not in seen_properties:
            final_top_properties.append(property_name)
            seen_properties.add(property_name)
        if len(final_top_properties) == num_properties:
            break

    return set(final_top_properties)
# # Example usage
# csv_string = """Alloy	Process	σy (GPa)	σmax (GPa)	εp (%)	Hardness (Hv)	Refs.
# """

# csv_string=convert_to_csv(csv_string)
# print(csv_string)
# retrieved_properties = process_csv_and_retrieve_properties(csv_string, store,num_properties=15)
# print(retrieved_properties)

Loaded existing vector store.


In [ ]:
from openai import OpenAI

client_2 = OpenAI(api_key='') # Gpt-4o-mini openai api key

import random
import time

max_retries = 5

def exponential_backoff(retry_count):
    """Calculates the exponential backoff time."""
    return min(2 ** retry_count + random.random(), 60)  # Cap the delay to 60 seconds

def generate_completion_gpt_4_mini(prompt):
    """Generates completion using the OpenAI API with retries and exponential backoff."""
    for retry in range(max_retries):
        try:
            response = client_2.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                stream=True,
                temperature=0
            )
            res = ''
            for chunk in response:
              if chunk.choices[0].delta.content is not None:
                  res += chunk.choices[0].delta.content
            return res
        except Exception as e:
            print(f"Error: {e}")
            if retry == max_retries - 1:
                raise
            sleep_time = exponential_backoff(retry)
            print(f"Retrying in {sleep_time:.2f} seconds...")
            time.sleep(sleep_time)

def generate_completion_gpt_4(prompt):
    """Generates completion using the OpenAI API with retries and exponential backoff."""
    for retry in range(max_retries):
        try:
            response = client_2.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                stream=True,
                temperature=0
            )
            res = ''
            for chunk in response:
              if chunk.choices[0].delta.content is not None:
                  res += chunk.choices[0].delta.content
            return res
        except Exception as e:
            print(f"Error: {e}")
            if retry == max_retries - 1:
                raise
            sleep_time = exponential_backoff(retry)
            print(f"Retrying in {sleep_time:.2f} seconds...")
            time.sleep(sleep_time)

In [8]:
def create_prompt(table_caption, table_footer, table_data,properties):
    prompt= f'''Instructions:
1. The context section contains definitions of various material properties labelled by an expert after going through tables in literature. These are the potential properties that you can find in the table along with some description that can aid you. The table section contains a table caption and a table. Your task is to identify the properties from the context section that are described in the table. If no property from the context section is described in the table, reply with a '-'. 
2. If one more properties from the context section is present in the table return a table in CSV format that contains the alloy composition, processing condition(if available in the table), testing condition(if available in the table) the property present in table(as written in the context section), property value and units as present in the table provided.
3. Note that the table may contain the same property as in the context but with a different naming convention. Use the context to understand the property and decide if any property from the context is described in the table. If some property is mentioned with a heading that you are unsure of from the context provided, do not extract that property. Extract only those properties available in the context.
4. Since you are extracting data in csv format, makes sure that each row of extraction has exactly 5 commas. Use semicolon if needed in the testing conditions(avoid use of commas within a cell as this is necessary to identify the csv table).
5. Refer to the examples and follow the above instructions strictly.

Context(the property column in the extracted table strictly be these property names only):

{properties}

Key Notes:
1. There are a lot of properties with clear definitions. In addition to the properties mentioned above, if you are sure that one of the following properties is present, extract them in the prescribed format: Density, Melting point, Boiling point, Shear modulus, Young's modulus(elastic modulus), Poisson's ratio.

2. Properties for a material cannot always be described through a number alone. For example, the Young's modulus of a material can vary with temperature and this is usually mentioned in the table details; be sure to extract this and report it as a testing condition. 

3. An alloy system is usually subjected to some processing before property is evaluated, a common example is arc-melting or as-casting an alloy at a particular temperature; be sure to extract this and report it as a testing condition.

4. Some properties like refractive index, Poisson's ratio do not have units, report units as "-".

5. Since material systems may have hundreds of property to describe the system, the most frequently occuring ones and listed in the above sections. However, if a table contains units, value and details of any other property mention it as "Others(<insert property name>)" in the Property column and report it with value and unit in the same format as other properties.

Examples of an expert performing your task(in addition to property names you have been given descriptions to aid you):
Go through these example prompts:
Example 1:
Table Caption: "Table 4. Shear Modulus(GPa) measured at various temperatures for 3 alloys."
Table Data:
Temperature (K), Pressure (bar),FeNiCoCr,FeNi,Ni
77,1,-,68,84
203,2,-,66,80
293,2.5,84,62,76
Context: Elastic Moduli, Shear Modulus, Ultimate Tensile Strength, Elasticity

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition, Testing condition, Property,Value,Unit
FeNiCoCr,-,Temperature(in K): 293 ;Pressure(in bar) 2.5,Shear Modulus,84,GPa
FeNi,-,Temperature(in K): 77 ;Pressure(in bar) 1,Shear Modulus,68,GPa
FeNi,-,Temperature(in K): 203 ;Pressure(in bar) 2,Shear Modulus,66,GPa
FeNi,-,Temperature(in K): 293 ;Pressure(in bar) 2.5,Shear Modulus,62,GPa
Ni,-,Temperature(in K): 77 ;Pressure(in bar) 1,Shear Modulus,84,GPa
Ni,-,Temperature(in K): 203 ;Pressure(in bar) 2,Shear Modulus,80,GPa
Ni,-,Temperature(in K): 293 ;Pressure(in bar) 2.5,Shear Modulus,76,GPa

Example 2:
Table Caption: "Table 4. Vickers microhardness of Al0.5CoCrCuFeNi alloys in four different states."
Table Data:
State,Micro-hardness (HV)
As-cast,208±2
As-forged,350±3
As-homogenized-FC,265±4
As-homogenized-WQ,154±4
Context: Hardness, Strain, Compressive Strength, Elastic Moduli

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition,Testing condition,Property,Value,Unit
Al0.5CoCrCuFeNi,As-cast,-,Hardness,208±2,HV
Al0.5CoCrCuFeNi,As-forged,-,Hardness,350±3,HV
Al0.5CoCrCuFeNi,As-homogenized-FC,-,Hardness,265±4,HV
Al0.5CoCrCuFeNi,As-homogenized-WQ,-,Hardness,154±4,HV

Example 3:
Table Caption: "Table 3. Summary of results on friction properties of hot-forged alloys tested"
Table Data:
Alloy peak coefficient of friction (µp)
Al0.5CoCrCuFeNi,0.65 ± 0.04
Al1.0CoCrCuFeNi,0.65 ± 0.03
Al2.0CoCrCuFeNi,0.70 ± 0.02
Context: Hardness, Lattice constants, Refractive index

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition,Testing condition,Property,Value,Unit
Al0.5CoCrCuFeNi,hot-forged,-,Others(coefficient of friction),0.65 ± 0.04,-
Al1.0CoCrCuFeNi,hot-forged,-,Others(coefficient of friction),0.65 ± 0.03,-
Al2.0CoCrCuFeNi,hot-forged,-,Others(coefficient of friction),0.70 ± 0.02,-

Example 4:
Table Caption: "Table 2. Chemical compositions of worn surface and wear debris of AlxCoCrCuFeNi alloys in atomic percentage"
Table Data:
x-Value,Condition,Al,Co,Cr,Cu,Fe,Ni,O
0.5,Nominal,9.09,18.18,18.18,18.18,18.18,18.18,
0.5,Worn surface,7.2,16.5,17.3,14.7,17.5,16.2,12.4
0.5,Wear debris,6.1,16.4,16.1,15.5,16.5,15.3,16.2
Context: Hardness, Lattice constants, Chemical composition

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition,Testing condition,Property,Value,Unit
Al0.5CoCrCuFeNi,-,Nominal,Chemical Composition,Al: 9.09; Co: 18.18; Cr: 18.18; Cu: 18.18; Fe: 18.18; Ni: 18.18,at%
Al0.5CoCrCuFeNi,-,Worn surface,Chemical Composition,Al: 7.2; Co: 16.5; Cr: 17.3; Cu: 14.7; Fe: 17.5; Ni: 16.2; O: 12.4,at%
Al0.5CoCrCuFeNi,-,Wear debris,Chemical Composition,Al: 6.1; Co: 16.4; Cr: 16.1; Cu: 15.5; Fe: 16.5; Ni: 15.3; O: 16.2,at%


Table:
Table Caption: "{table_caption}"
Table Data:\n{table_data}

Table Footer: "{table_footer if table_footer else ''}"
Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Output: '''
     
    return prompt

In [1]:
def create_prompt1(table,properties):
    prompt= f'''Instructions:
1. The context section contains definitions of various material properties labelled by an expert after going through tables in literature. These are the potential properties that you can find in the table along with some description that can aid you. The table section contains a table caption and a table. Your task is to identify the properties from the context section that are described in the table. If no property from the context section is described in the table, reply with a '-'. 
2. If one more properties from the context section is present in the table return a table in CSV format that contains the alloy composition, processing condition(if available in the table), testing condition(if available in the table) the property present in table(as written in the context section), property value and units as present in the table provided.
3. Note that the table may contain the same property as in the context but with a different naming convention. Use the context to understand the property and decide if any property from the context is described in the table. If some property is mentioned with a heading that you are unsure of from the context provided, do not extract that property. Extract only those properties available in the context.
4. Since you are extracting data in csv format, makes sure that each row of extraction has exactly 5 commas. Use semicolon if needed in the testing conditions(avoid use of commas within a cell as this is necessary to identify the csv table).
5. Refer to the examples and follow the above instructions strictly.

Context(the property column in the extracted table strictly be these property names only):

{properties}

Key Notes:
1. There are a lot of properties with clear definitions. In addition to the properties mentioned above, if you are sure that one of the following properties is present, extract them in the prescribed format: Density, Melting point, Boiling point, Shear modulus, Young's modulus(elastic modulus), Poisson's ratio.

2. Properties for a material cannot always be described through a number alone. For example, the Young's modulus of a material can vary with temperature and this is usually mentioned in the table details; be sure to extract this and report it as a testing condition. 

3. An alloy system is usually subjected to some processing before property is evaluated, a common example is arc-melting or as-casting an alloy at a particular temperature; be sure to extract this and report it as a testing condition.

4. Some properties like refractive index, Poisson's ratio do not have units, report units as "-".

5. Since material systems may have hundreds of property to describe the system, the most frequently occuring ones and listed in the above sections. However, if a table contains units, value and details of any other property mention it as "Others(<insert property name>)" in the Property column and report it with value and unit in the same format as other properties.

Examples of an expert performing your task(in addition to property names you have been given descriptions to aid you):
Go through these example prompts:
Example 1:
Table Caption: "Table 4. Shear Modulus(GPa) measured at various temperatures for 3 alloys."
Table Data:
Temperature (K), Pressure (bar),FeNiCoCr,FeNi,Ni
77,1,-,68,84
203,2,-,66,80
293,2.5,84,62,76
Context: Elastic Moduli, Shear Modulus, Ultimate Tensile Strength, Elasticity

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition, Testing condition, Property,Value,Unit
FeNiCoCr,-,Temperature(in K): 293 ;Pressure(in bar) 2.5,Shear Modulus,84,GPa
FeNi,-,Temperature(in K): 77 ;Pressure(in bar) 1,Shear Modulus,68,GPa
FeNi,-,Temperature(in K): 203 ;Pressure(in bar) 2,Shear Modulus,66,GPa
FeNi,-,Temperature(in K): 293 ;Pressure(in bar) 2.5,Shear Modulus,62,GPa
Ni,-,Temperature(in K): 77 ;Pressure(in bar) 1,Shear Modulus,84,GPa
Ni,-,Temperature(in K): 203 ;Pressure(in bar) 2,Shear Modulus,80,GPa
Ni,-,Temperature(in K): 293 ;Pressure(in bar) 2.5,Shear Modulus,76,GPa

Example 2:
Table Caption: "Table 4. Vickers microhardness of Al0.5CoCrCuFeNi alloys in four different states."
Table Data:
State,Micro-hardness (HV)
As-cast,208±2
As-forged,350±3
As-homogenized-FC,265±4
As-homogenized-WQ,154±4
Context: Hardness, Strain, Compressive Strength, Elastic Moduli

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition,Testing condition,Property,Value,Unit
Al0.5CoCrCuFeNi,As-cast,-,Hardness,208±2,HV
Al0.5CoCrCuFeNi,As-forged,-,Hardness,350±3,HV
Al0.5CoCrCuFeNi,As-homogenized-FC,-,Hardness,265±4,HV
Al0.5CoCrCuFeNi,As-homogenized-WQ,-,Hardness,154±4,HV

Example 3:
Table Caption: "Table 3. Summary of results on friction properties of hot-forged alloys tested"
Table Data:
Alloy peak coefficient of friction (µp)
Al0.5CoCrCuFeNi,0.65 ± 0.04
Al1.0CoCrCuFeNi,0.65 ± 0.03
Al2.0CoCrCuFeNi,0.70 ± 0.02
Context: Hardness, Lattice constants, Refractive index

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition,Testing condition,Property,Value,Unit
Al0.5CoCrCuFeNi,hot-forged,-,Others(coefficient of friction),0.65 ± 0.04,-
Al1.0CoCrCuFeNi,hot-forged,-,Others(coefficient of friction),0.65 ± 0.03,-
Al2.0CoCrCuFeNi,hot-forged,-,Others(coefficient of friction),0.70 ± 0.02,-

Example 4:
Table Caption: "Table 2. Chemical compositions of worn surface and wear debris of AlxCoCrCuFeNi alloys in atomic percentage"
Table Data:
x-Value,Condition,Al,Co,Cr,Cu,Fe,Ni,O
0.5,Nominal,9.09,18.18,18.18,18.18,18.18,18.18,
0.5,Worn surface,7.2,16.5,17.3,14.7,17.5,16.2,12.4
0.5,Wear debris,6.1,16.4,16.1,15.5,16.5,15.3,16.2
Context: Hardness, Lattice constants, Chemical composition

Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Expected Output:
Alloy,Processing condition,Testing condition,Property,Value,Unit
Al0.5CoCrCuFeNi,-,Nominal,Chemical Composition,Al: 9.09; Co: 18.18; Cr: 18.18; Cu: 18.18; Fe: 18.18; Ni: 18.18,at%
Al0.5CoCrCuFeNi,-,Worn surface,Chemical Composition,Al: 7.2; Co: 16.5; Cr: 17.3; Cu: 14.7; Fe: 17.5; Ni: 16.2; O: 12.4,at%
Al0.5CoCrCuFeNi,-,Wear debris,Chemical Composition,Al: 6.1; Co: 16.4; Cr: 16.1; Cu: 15.5; Fe: 16.5; Ni: 15.3; O: 16.2,at%

Table:
{table}
Question: Extract the properties of alloy systems present in the context from the following table data. Provide the data in comma-separated values (CSV) format only.
Output: '''
     
    return prompt

In [73]:
table="""Table 1. Chemical compositions (in at.%), density, ρ, and Vickers microhardness, Hv, of the alloys studied in this work.

Alloy	Cr	Nb	Ti	V	Zr	ρ (g cm−3)	ρmix (g cm−3)	HV (GPa)
NbTiVZr	–	28.3	24.5	23.0	24.2	6.52	6.50	3.29 ± 0.03
NbTiV2Zr	–	22.6	19.4	37.2	20.8	6.34	6.38	2.99 ± 0.03
CrNbTiZr	24.6	26.7	23.9	–	24.8	6.67	6.70	4.10 ± 0.04
CrNbTiVZr	20.2	20.0	19.9	19.6	20.3	6.57	6.52	4.72 ± 0.05

"""
table_data_list=table.split("\n\n")
print(table_data_list)
table_caption=table_data_list[0]
print(table_caption)
table_data=table_data_list[1]
print(table_data)

['Table 1. Chemical compositions (in at.%), density, ρ, and Vickers microhardness, Hv, of the alloys studied in this work.', 'Alloy\tCr\tNb\tTi\tV\tZr\tρ (g cm−3)\tρmix (g cm−3)\tHV (GPa)\nNbTiVZr\t–\t28.3\t24.5\t23.0\t24.2\t6.52\t6.50\t3.29 ± 0.03\nNbTiV2Zr\t–\t22.6\t19.4\t37.2\t20.8\t6.34\t6.38\t2.99 ± 0.03\nCrNbTiZr\t24.6\t26.7\t23.9\t–\t24.8\t6.67\t6.70\t4.10 ± 0.04\nCrNbTiVZr\t20.2\t20.0\t19.9\t19.6\t20.3\t6.57\t6.52\t4.72 ± 0.05', '']
Table 1. Chemical compositions (in at.%), density, ρ, and Vickers microhardness, Hv, of the alloys studied in this work.
Alloy	Cr	Nb	Ti	V	Zr	ρ (g cm−3)	ρmix (g cm−3)	HV (GPa)
NbTiVZr	–	28.3	24.5	23.0	24.2	6.52	6.50	3.29 ± 0.03
NbTiV2Zr	–	22.6	19.4	37.2	20.8	6.34	6.38	2.99 ± 0.03
CrNbTiZr	24.6	26.7	23.9	–	24.8	6.67	6.70	4.10 ± 0.04
CrNbTiVZr	20.2	20.0	19.9	19.6	20.3	6.57	6.52	4.72 ± 0.05


In [ ]:
import openai  # Ensure you have the `openai` package installed

table="""Table 4. Resonance fields, g-values and amplitudes of the ESR-spectra of the investigated alloys with H in the sample plane (θ = 0°).

Alloy	Hres in G	g-value	ΔH in G	Amplitude in A
CrMnFeCoNi	HFMR=500
2900	-
2.317	1500
800	120
110
CrFeCoNi	HFMR=200
2500	-
2.689	1500
350	2212
2528
CrFeNi	3150	2.135	560	89
CrCoNi	3136
3267
3501
CESR: 3788.6	2.144
2.058
1.92
1.77	157
835
130
522	18
11
14
26
"""
table_data_list=table.split("\n\n")
table_caption=table_data_list[0]
table_data=table_data_list[1]
# table_caption="Table 1. Chemical composition of the TaNbHfZrTi alloy produced by vacuum arc melting."
# table_data="""Composition	Ta	Nb	Hf	Zr	Ti
# at.%	19.68	18.93	20.46	21.23	19.7
# wt.%	30.04	14.84	30.82	16.34	7.96
# """
table_footer=""

prompt,r,c=convert_to_csv(table)
print(prompt)
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a materials science expert identifying cells in a table that have material property names. The table title and footer too may contain property name include them too. You must respond with cells that you think contain a property name in a comma separated format. Make sure each of the properties in your response mentioned are unique."},
        {"role": "user", "content": f"{prompt}"}
    ]
)

description = response.choices[0].message.content.strip()
# print(description)
description=normalize_query(description)
print(description)
retrieved_properties = process_csv_and_retrieve_properties(description, store,num_properties=10)
print(retrieved_properties)


import pandas as pd

# Load CSV file
df = pd.read_csv("final_properties_with_descriptions.csv")

# Filter the dataframe to only include rows where "Property" is in the set
# Filter the dataframe to only include rows where "Property" is in the set
filtered_df = df[df["Property"].isin(retrieved_properties)]

# Reset index for numbering
filtered_df = filtered_df.reset_index(drop=True)

# Create the final string with numbering
properties = "\n\n".join(f"{i+1}. {row['Property']}: {row['Description']}" for i, row in filtered_df.iterrows())


prompt=create_prompt1(normalize_query(table),normalize_query(properties))
# print(prompt)
if (r>=6 or c>=5) and r*c>=25:
    response=generate_completion_gpt_4(prompt)
else:
    response=generate_completion_gpt_4_mini(prompt)
print(response)

import os

def append_csv(csv_string, filename="test.csv"):
    lines = csv_string.strip().split("\n")
    header, data = lines[0], lines[1:]  # Split header and data

    file_exists = os.path.exists(filename)

    with open(filename, "a", encoding="utf-8") as file:
        if not file_exists:
            file.write(header + "\n")  # Write header only if the file does not exist
        else:
            with open(filename, "r", encoding="utf-8") as existing_file:
                existing_header = existing_file.readline().strip()
            if existing_header != header:
                raise ValueError("Header mismatch! Ensure all CSV inputs have the same header.")

        file.write("\n".join(data) + "\n")  # Append new data
if response!="-":
    append_csv(response)

"Table 4. Resonance fields, g-values and amplitudes of the ESR-spectra of the investigated alloys with H in the sample plane (θ = 0°)."
""
Alloy,Hres in G,g-value,ΔH in G,Amplitude in A
CrMnFeCoNi,HFMR=500
2900,-
2.317,1500
800,120
110
CrFeCoNi,HFMR=200
2500,-
2.689,1500
350,2212
2528
CrFeNi,3150,2.135,560,89
CrCoNi,3136
3267
3501
CESR: 3788.6,2.144
2.058
1.92
1.77,157
835
130
522,18
11
14
26



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Resonance fields, g-values, Amplitude, DeltaH


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'Elasticity', 'Sound Velocity', 'Stress', 'Phase Transition', 'Residual Stress', 'Magnetostriction', 'Grüneisen Coefficient', 'Stress and Strain', 'Elastic Moduli', 'Remanence'}


NameError: name 'create_prompt1' is not defined

In [ ]:
import os

# Load TestData.xlsx
df = pd.read_excel("TestData.xlsx", engine="openpyxl")  

# Ensure logging file exists
log_filename = "Log_Test.log"
if not os.path.exists(log_filename):
    with open(log_filename, "w", encoding="utf-8") as log_file:
        log_file.write("Processing Log\n\n")  # CSV Header

# Function to log details in a structured format
def log_details(index, normalized_table, retrieved_properties, output, error=False):
    with open(log_filename, "a", encoding="utf-8") as log_file:
        log_file.write(f"Table Index: {index}\n")
        if error:
            log_file.write(f"ERROR: {normalized_table}\n")
        else:
            log_file.write(f"Normalized Table:\n{normalized_table}\n\n")
            log_file.write(f"Retrieved Properties:\n{retrieved_properties}\n\n")
            log_file.write(f"Output:\n{output}\n")
        log_file.write("\n" + "="*50 + "\n\n")  # Separator for readability

# Function to process table text and get properties
def process_table(table_text, index):
    try:
        # Convert table to CSV format
        prompt, r, c = convert_to_csv(table_text)  
        
        # Get property names using GPT
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a materials science expert identifying cells in a table that have material property names. The table title and footer too may contain property names. Include them too. You must respond with cells that you think contain a property name in a comma-separated format. Make sure each of the properties in your response mentioned are unique."},
                {"role": "user", "content": f"{prompt}"}
            ]
        )

        description = response.choices[0].message.content.strip()
        description = normalize_query(description)

        # Retrieve detailed property descriptions
        retrieved_properties = process_csv_and_retrieve_properties(description, store, num_properties=10)

        # Load property descriptions from CSV
        df_properties = pd.read_csv("final_properties_with_descriptions.csv")
        filtered_df = df_properties[df_properties["Property"].isin(retrieved_properties)].reset_index(drop=True)

        # Format response
        properties = "\n\n".join(f"{i+1}. {row['Property']}: {row['Description']}" for i, row in filtered_df.iterrows())

        # Generate final prompt and get model response
        normalized_table = normalize_query(table_text)
        final_prompt = create_prompt1(normalized_table, normalize_query(properties))

        if (r >= 6 or c >= 5) and (r * c >= 25):
            output = generate_completion_gpt_4(final_prompt)
        else:
            output = generate_completion_gpt_4_mini(final_prompt)

        # Log structured data
        log_details(index, normalized_table, retrieved_properties, output)

        return output

    except Exception as e:
        # Log error if processing fails
        error_message = f"Error processing table {index}: {str(e)}"
        log_details(index, error_message, "", "", error=True)
        print(error_message)
        return "-"


Error processing table 0: name 'create_prompt1' is not defined
Error processing table 1: name 'create_prompt1' is not defined
Error processing table 2: name 'create_prompt1' is not defined
Error processing table 3: name 'create_prompt1' is not defined
Error processing table 4: name 'create_prompt1' is not defined
Error processing table 5: name 'create_prompt1' is not defined
Error processing table 6: name 'create_prompt1' is not defined


KeyboardInterrupt: 

In [9]:
# Function to append results to CSV
def append_csv(csv_string, filename="Results_Test.csv"):
    lines = csv_string.strip().split("\n")
    header, data = lines[0], lines[1:]

    file_exists = os.path.exists(filename)

    with open(filename, "a", encoding="utf-8") as file:
        if not file_exists:
            file.write(header + "\n")
        else:
            with open(filename, "r", encoding="utf-8") as existing_file:
                existing_header = existing_file.readline().strip()
            if existing_header != header:
                raise ValueError("Header mismatch! Ensure all CSV inputs have the same header.")

        file.write("\n".join(data) + "\n")


print("Processing complete. Results saved in Results_Test.csv and log saved in Log_Test.log.")

Processing complete. Results saved in Results_Test.csv and log saved in Log_Test.log.


In [ ]:
import logging
from io import StringIO
import logging
import httpx

logging.getLogger("httpx").setLevel(logging.WARNING)
# Function to process each table CSV
def process_table(table_path, table_caption, table_footer):
    try:
        df = pd.read_csv(table_path)
        df = df.astype(str) 
        table_data = df.to_csv(index=False)
        # print(type(table_data))
        df_temp = pd.read_csv(StringIO(table_data))
        # Get row and column count
        r, c = df_temp.shape
        # Convert to CSV format as expected in your pipeline
        # prompt, r, c = convert_to_csv(table_data)
        # print(type(prompt))
        table_caption = str(table_caption) if not isinstance(table_caption, str) else table_caption
        table_footer = str(table_footer) if not isinstance(table_footer, str) else table_footer
        table=f"""Table caption: {normalize_query(table_caption)}
        Table Data: {normalize_query(table_data)}
        Table footer: {normalize_query(table_footer)}
        """
        print(table)
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a materials science expert identifying cells in a table that have material property names. The table caption and footer too may contain property names. You must respond with cells that you think contain a property name in a comma-separated format. Make sure each of the properties in your response mentioned are unique."},
                {"role": "user", "content": f"{table}"}
            ]
        )
        description = response.choices[0].message.content.strip()
        description = normalize_query(description)
        print(description)
        retrieved_properties = process_csv_and_retrieve_properties(description, store, num_properties=10)
        print(retrieved_properties)
        # Load property descriptions
        df_properties = pd.read_csv("final_properties_with_descriptions.csv")
        filtered_df = df_properties[df_properties["Property"].isin(retrieved_properties)]
        filtered_df = filtered_df.reset_index(drop=True)

        properties = "\n\n".join(f"{i+1}. {row['Property']}: {row['Description']}" for i, row in filtered_df.iterrows())

        # Generate final prompt
        prompt = create_prompt(normalize_query(table_caption), normalize_query(table_footer), normalize_query(table_data), normalize_query(properties))

        # Determine model based on table size
        if (r >= 6 or c >= 5) and r * c >= 25:
            final_response = generate_completion_gpt_4(prompt)
        else:
            final_response = generate_completion_gpt_4_mini(prompt)
        print(final_response)
        return table_data, retrieved_properties, final_response
        return

    except Exception as e:
        logging.error(f"Error processing table {table_path}: {str(e)}")
        return None, None, None

# Function to process all tables in a given DOI folder
def process_doi_folder(doi_folder, final_output_file="final_outputs_testset2.csv"):
    tables_folder = os.path.join(doi_folder, "tables")
    metadata_path = os.path.join(doi_folder, "Table_Metadata.csv")
    log_path = os.path.join(doi_folder, "processing_log.txt")

    if not os.path.exists(metadata_path):
        logging.error(f"Metadata file missing for {doi_folder}")
        return

    # Load table metadata
    metadata_df = pd.read_csv(metadata_path)

    with open(log_path, "w", encoding="utf-8") as log_file:
        for _, row in metadata_df.iterrows():
            table_label = row["Table Label"]
            table_caption = row["Table Caption"]
            table_footer = row["Table Footer"]
            table_path = os.path.join(tables_folder, f"{table_label}.csv")

            if not os.path.exists(table_path):
                logging.warning(f"Table file {table_path} missing.")
                continue

            # Process table
            table_data, retrieved_properties, final_output = process_table(table_path, table_caption, table_footer)

            if table_data is None or retrieved_properties is None or final_output is None:
                log_file.write(f"\nTable {table_label}: Error processing table.\n")
                continue

            # Log results
            log_file.write(f"\nTable {table_label}:\n")
            log_file.write(f"Input Table:\n{table_data}\n")
            log_file.write(f"Retrieved Properties: {', '.join(retrieved_properties)}\n")
            log_file.write(f"Final Output:\n{final_output}\n")

            # Append results to final_outputs.csv
            if final_output != "-":
                append_csv(final_output, final_output_file)

# Process all DOI folders in the dataset
def process_all_dois(root_folder="articles_16_march2/articles_16_march2"):
    doi_folders = [os.path.join(root_folder, d) for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

    for doi_folder in doi_folders:
        process_doi_folder(doi_folder)

if __name__ == "__main__":
    logging.basicConfig(level=logging.WARNING)
    process_all_dois()

Table caption: Chemical compositions (in at.%), density, r, and Vickers microhardness, H
v, of the alloys studied in this work.
        Table Data: Alloy,Cr,Nb,Ti,V,Zr,r (gcm-3),r mix (gcm-3),H V (GPa)
NbTiVZr,-,28.3,24.5,23.0,24.2,6.52,6.5,3.29+-0.03
NbTiV2Zr,-,22.6,19.4,37.2,20.8,6.34,6.38,2.99+-0.03
CrNbTiZr,24.6,26.7,23.9,-,24.8,6.67,6.7,4.10+-0.04
CrNbTiVZr,20.2,20.0,19.9,19.6,20.3,6.57,6.52,4.72+-0.05

        Table footer: nan
        
density, Vickers microhardness
{'Relative Density', 'Density', 'Mass Density', 'Hardness', 'Indentation Properties'}
Alloy,Processing condition,Testing condition,Property,Value,Unit
NbTiVZr,-,-,Density,6.52,gcm-3
NbTiV2Zr,-,-,Density,6.34,gcm-3
CrNbTiZr,-,-,Density,6.67,gcm-3
CrNbTiVZr,-,-,Density,6.57,gcm-3
NbTiVZr,-,-,Hardness,3.29±0.03,GPa
NbTiV2Zr,-,-,Hardness,2.99±0.03,GPa
CrNbTiZr,-,-,Hardness,4.10±0.04,GPa
CrNbTiVZr,-,-,Hardness,4.72±0.05,GPa
Table caption: Chemical composition (in at.%) of the NbVTiZr alloy constituents (see Fig. 2).
     

ERROR:root:Metadata file missing for articles_16_march2/articles_16_march2\10.1016.j.jallcom.2017.08.067


Alloy,Processing condition,Testing condition,Property,Value,Unit
AlNbTiV,-,Temperature(in degC): 600;Phase: BCC_A2,Phase Fractions,0.697,-
AlNbTiV,-,Temperature(in degC): 600;Phase: Ti3Al,Phase Fractions,0.303,-
AlNbTiV,-,Temperature(in degC): 800;Phase: BCC_A2,Phase Fractions,1,-
AlNbTiV,-,Temperature(in degC): 1000;Phase: BCC_A2,Phase Fractions,1,-
AlNbTiV,-,Temperature(in degC): 1200;Phase: BCC_A2,Phase Fractions,1,-
AlCr0.5NbTiV,-,Temperature(in degC): 600;Phase: BCC_A2,Phase Fractions,0.701,-
AlCr0.5NbTiV,-,Temperature(in degC): 600;Phase: Laves,Phase Fractions,0.135,-
AlCr0.5NbTiV,-,Temperature(in degC): 600;Phase: Ti3Al,Phase Fractions,0.164,-
AlCr0.5NbTiV,-,Temperature(in degC): 800;Phase: BCC_A2,Phase Fractions,1,-
AlCr0.5NbTiV,-,Temperature(in degC): 1000;Phase: BCC_A2,Phase Fractions,1,-
AlCr0.5NbTiV,-,Temperature(in degC): 1200;Phase: BCC_A2,Phase Fractions,1,-
AlCrNbTiV,-,Temperature(in degC): 600;Phase: BCC_A2,Phase Fractions,0.636,-
AlCrNbTiV,-,Temperature(in degC): 600;

In [ ]:
# Read the CSV file with proper handling of quotes and delimiters
with open("TestData.csv", "r", encoding="utf-8", errors="ignore") as f:
    df = pd.read_csv(f,delimiter=",", quotechar='"')

# Verify column names to ensure correct parsing
print(df.columns)

# Try accessing "Tables" column correctly
if "Tables" in df.columns:
    for index, row in df.iterrows():
        table_text = row["Tables"]  # Extract table text
        print(f"Processing Table {index+1}:\n{table_text}\n")
else:
    print("Error: 'Tables' column not found! Check CSV formatting.")

Index(['Table 1. Chemical composition of the AlCoCrCuNi and AlCoCrCuFeNi alloys in atomic percentage\n\nAlloy\tAl\tCo\tCr\tCu\tFe\tNi\nAlCoCrCuNi\n Nominal\t20\t20\t20\t20\t\t20\n Dendrite\t21\t21\t25\t11\t\t22\n Interdendrite\t13\t8\t6\t58\t\t15\nAlCoCrCuFeNi\n Nominal\t17\t17\t17\t17\t17\t17\n Dendrite\t17\t17\t21\t11\t18\t16\n Interdendrite\t12\t5\t3\t63\t5\t12\n', 'https://doi.org/10.1016/j.msea.2007.01.122'], dtype='object')
Error: 'Tables' column not found! Check CSV formatting.


In [26]:
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 21.2.4 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Load the Excel file
df = pd.read_excel("TestData.xlsx", engine="openpyxl")  # Ensure openpyxl is installed

# Display the first few rows to check the structure
print(df.head())

# Verify column names
print(df.columns)

                                              Tables  \
0  Table 1. Chemical composition of the AlCoCrCuN...   
1  Table 2. Hardness of the AlCoCrCuNi, AlCoCrCuF...   
2  Table 1. Chemical compositions of cast AlxCoCr...   
3  Table 2. Chemical compositions of worn surface...   
4  Table 3. Summary of results on friction proper...   

                                                 DOI  
0         https://doi.org/10.1016/j.msea.2007.01.122  
1         https://doi.org/10.1016/j.msea.2007.01.122  
2  https://www.sciencedirect.com/science/article/...  
3  https://www.sciencedirect.com/science/article/...  
4  https://www.sciencedirect.com/science/article/...  
Index(['Tables', 'DOI'], dtype='object')
